In [ ]:
import azureml.core
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2V3', 
                                                                max_nodes=6)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

In [ ]:
import os

project_folder = './experiment'
os.makedirs(project_folder, exist_ok=True)
import shutil
shutil.copytree('common', project_folder + '/common')
shutil.copytree('adapters', project_folder + '/adapters')
shutil.copy('pytorch_train.py', project_folder)


In [ ]:
from azureml.core import Experiment

experiment_name = 'mnist'
experiment = Experiment(ws, name=experiment_name)

In [ ]:
from common import context
#Fill your app credentials here
app_context = context.Context(account_name = '', \
    account_key = '', \
    container = '', \
    folder = '')    


In [ ]:
from azureml.train.dnn import PyTorch

estimator = PyTorch(source_directory=project_folder, 
                    script_params={'--batch-size': '8',
                                    '--account-name': app_context.account_name,
                                    '--account-key': app_context.account_key,
                                    '--container': app_context.container,
                                    '--folder': app_context.folder,
                                    '--start-date': '2020-02-29',
                                    '--end_date': '2020-03-01'},
                    compute_target=compute_target,
                    entry_script='pytorch_train.py',
                    use_gpu=True,
                    pip_packages=['azure-storage-blob', 'azure-storage'])

# upgrade to PyTorch 1.0 Preview, which has better support for ONNX
estimator.conda_dependencies.remove_conda_package('pytorch-0.4.0')
estimator.conda_dependencies.add_conda_package('pytorch-nightly')
estimator.conda_dependencies.add_conda_package('torchvision')
estimator.conda_dependencies.add_channel('pytorch')

In [ ]:
run = experiment.submit(estimator)
print(run.get_details())

In [ ]:
model_path_exp = os.path.join('outputs', 'current.onnx')
model_path_local = '/Users/ataymano/data/byom/downloaded/current.onnx'
run.download_file(model_path_exp, output_file_path=model_path_local)
print('Model is downloaded')
 
app_context.deploy_model(model_path_local)
print('Model is deployed')
